In [1]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams.update({'font.size': 12})
pylab.rcParams.update({'font.family': 'sans serif'})
pylab.rcParams.update({'mathtext.default':'regular'})
pylab.rcParams.update({'mathtext.fontset':'stixsans'})
pylab.rcParams.update({'axes.linewidth': 1.5})
pylab.rcParams.update({'xtick.direction':'in'})
pylab.rcParams.update({'xtick.major.size': 5})
pylab.rcParams.update({'xtick.major.width': 1.25 })
pylab.rcParams.update({'xtick.minor.size': 2.5})
pylab.rcParams.update({'xtick.minor.width': 1.25 })
pylab.rcParams.update({'ytick.direction':'in'})
pylab.rcParams.update({'ytick.major.size': 5})
pylab.rcParams.update({'ytick.major.width': 1.25 })
pylab.rcParams.update({'ytick.minor.size': 2.5})
pylab.rcParams.update({'ytick.minor.width': 1.25 })

import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
class AccretionIntegrator:
    # conversion factors to get CGS units into Cholla units
    MP = 1.6726E-24;  # proton mass in grams
    KB = 1.380658e-16; # Boltzmann constant
    LENGTH_UNIT = 3.08567758e21; # kpc in cm
    TINY_NUMBER = 1.0e-20;
    MASS_UNIT = 1.98855e33; # solar mass in grams
    TIME_UNIT = 3.15569e10; # kyr in s
    DENSITY_UNIT = (MASS_UNIT/(LENGTH_UNIT*LENGTH_UNIT*LENGTH_UNIT)); 
    VELOCITY_UNIT = (LENGTH_UNIT/TIME_UNIT);
    PRESSURE_UNIT = (DENSITY_UNIT*VELOCITY_UNIT*VELOCITY_UNIT);
    
    # solar abundances (percentage of total mass)
    # O, C, N, Si, Mg, Ne, Fe, S
    metals = np.array([0.0097, 0.0040, 0.00096, 0.00099, 0.00076, 0.00058, 0.0014, 0.00040]);
    metallicity = np.sum(metals)
    
    def __init__(self, T, n, dt):
        self.T = T
        self.n = n
        self.dt = dt
        self._d0_gas, self._d0_metal, self._d0_dust = self.calc_init_density()
        self.d_gas, self.d_metal, self.d_dust = self.calc_init_density()
        self.tau_g = self.calc_tau_g()

    def calc_tau_g(self):
        tau_g_ref = 0.2e-6 * self.TIME_UNIT; # 0.2 Gyr
        d_ref = self.MP * self.LENGTH_UNIT ** 3 / self.MASS_UNIT # 1 H atom per cubic centimer
        T_ref = 20.0;

        tau_g = tau_g_ref * (d_ref/(self._d0_gas+self._d0_dust)) * (T_ref/self.T) ** (1/2);

        return tau_g
    
    def calc_init_density(self):
        d0_gas = self.MP * self.n * self.LENGTH_UNIT ** 3 / self.MASS_UNIT
        d0_metal = self.metallicity * d0_gas
        # assume 1% dust to gas fraction
        d0_dust = d0_gas / 100
        
        return d0_gas, d0_metal, d0_dust

    def calc_dd_dt(self):
        dd_dt = (1 - self.d_dust/self.d_metal)*(self.d_dust/self.tau_g)
        
        return dd_dt
    
    def calc_d_dust(self):
        dd_dt = self.calc_dd_dt()
        d_dust_i = self.d_dust + self.dt * dd_dt
        
        return d_dust_i
    
    def update_densities(self):
        dd_dt = self.calc_dd_dt()
        self.d_dust += self.dt * dd_dt
        self.d_metal -= self.dt * dd_dt
    
    def get_info(self):
        print("Gas temperature: {:.1E} K".format(self.T))
        print("Gas initial density: {:.5E} M_sun/kpc^3".format(self._d0_gas))
        print("Growth timescale: {:.5E} kyr".format(self.tau_g))
        print("Time-step: {} yr".format(self.dt))
        print("------------------------------------------")

In [3]:
class SputteringIntegrator:
    # conversion factors to get CGS units into Cholla units
    MP = 1.6726E-24;  # proton mass in grams
    KB = 1.380658e-16; # Boltzmann constant
    LENGTH_UNIT = 3.08567758e21; # kpc in cm
    TINY_NUMBER = 1.0e-20;
    MASS_UNIT = 1.98855e33; # solar mass in grams
    TIME_UNIT = 3.15569e10; # kyr in s
    DENSITY_UNIT = (MASS_UNIT/(LENGTH_UNIT*LENGTH_UNIT*LENGTH_UNIT)); 
    VELOCITY_UNIT = (LENGTH_UNIT/TIME_UNIT);
    PRESSURE_UNIT = (DENSITY_UNIT*VELOCITY_UNIT*VELOCITY_UNIT);
    
    def __init__(self, T, n, dt):
        self.T = T
        self.n = n
        self.dt = dt
        self._d0_gas, self._d0_dust = self.calc_init_density()
        self.d_gas, self.d_dust = self.calc_init_density()
        self.tau_sp = self.calc_tau_sp()

    def calc_tau_sp(self):
        a1 = 3;
        d0 = 1;
        T_0 = 2e6;
        omega = 2.5;
        A = 1.7e5;
        
        tau_sp = A * (a1/d0) * ((T_0/self.T) ** omega + 1);
        
        return tau_sp
    
    def calc_init_density(self):
        d0_gas = self.MP * self.n * self.LENGTH_UNIT ** 3 / self.MASS_UNIT
        # assume 1% dust to gas fraction
        d0_dust = d0_gas / 100
        
        return d0_gas, d0_dust

    def calc_dd_dt(self):
        dd_dt = -self.d_dust/(self.tau_sp/3)
        
        return dd_dt
    
    def update_densities(self):
        dd_dt = self.calc_dd_dt()
        self.d_dust += self.dt * dd_dt
    
    def get_info(self):
        print("Sputtering")
        print("------------------------------------------")
        print("Gas temperature: {:.1E} K".format(self.T))
        print("Gas initial density: {:.5E} M_sun/kpc^3".format(self._d0_gas))
        print("Growth timescale: {:.5E} kyr".format(self.tau_sp))
        print("Time-step: {} yr".format(self.dt))
        print("------------------------------------------")

In [ ]:
def evolve_solutions(h, n, T_arr, T_colors):
    t_arr = np.arange(0, 1e7, h)
    
    d_dust = np.zeros(shape=(len(T_arr), len(t_arr)))
    d_metal = np.zeros(shape=(len(T_arr), len(t_arr)))

    acc_integrator = None
    sput_integrator = None
    
    def time_refine(integrator):
        d_dust_i = integrator.calc_d_dust()
        dd_dt = integrator.calc_dd_dt()
        dt = integrator.dt
        dd = dd_dt * dt
        while dd/d_dust_i > 0.01:
            dt_sub = 0.01 * d_dust_i / dd_dt

            integrator.d_dust += dt_sub * dd_dt
            integrator.d_metal -= dt_sub * dd_dt
            integrator.dt -= dt_sub
            dt = integrator.dt
            dd_dt = integrator.calc_dd_dt()
            dd = dt * dd_dt

    for j, T in enumerate(T_arr):
        integrator = AccretionIntegrator(T, n, h)
        integrator.get_info()
        # set initial densities
        d_dust[j][0] = integrator._d0_dust
        d_metal[j][0] = integrator._d0_metal

        for i, t_i in enumerate(t_arr):
            if (i+1) < len(t_arr):
                d_dust_i = integrator.calc_d_dust()
                dd_dt = integrator.calc_dd_dt()
                dd = integrator.dt * dd_dt
                
                # if rate of dust growth is changing too rapidly
                if dd/d_dust_i > 0.01:
                    # shorten time-step
                    time_refine(integrator)
                    integrator.dt = h
                    d_dust[j][i+1] = integrator.d_dust
                    d_metal[j][i+1] = integrator.d_metal
                    continue

                integrator.update_densities()
                d_dust[j][i+1] = integrator.d_dust
                d_metal[j][i+1] = integrator.d_metal

    return d_dust, d_metal, integrator